# Data Preprocessing

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

Next, we actually need to import our data. Since our data is stored as a CSV (comma seperated value) file, we will use the read_csv function provdided by pandas, which takes only our file name as an argument.

In [2]:
df = pd.read_csv('../../datasets/sample.csv')
df.head()

,Country,Age,Salary,Purchased
0,France,44.0,72000.0,No
1,Spain,27.0,48000.0,Yes
2,Germany,30.0,54000.0,No
3,Spain,38.0,61000.0,No
4,Germany,40.0,NaN,Yes


In [3]:
df.describe()

,Age,Salary
count,9.000000,9.000000
mean,38.777778,63777.777778
std,7.693793,12265.579662
min,27.000000,48000.000000
25%,35.000000,54000.000000
50%,38.000000,61000.000000
75%,44.000000,72000.000000
max,50.000000,83000.000000


This is our full set of data, but we need to split it up into our features and our response.

This is like the independent and dependent variables that one will have encountered in any introductory science course.
Our features will be used to build the model, which attempts to predict the response. 
Country, Age, and Salary are the features, which will attempt to predict **Purchased**, which, as in many machine learning models, is binary ("yes" or "no"). The *iloc* function is part of pandas and is used as an indexing tool for csv datasets. The general syntax is the index of the rows and then the columns, and take the form 'x:y'. The way indexing works in python is that it will take all data starting at the value given before the colon up to but not including the value given after the colon. For example, '1:3' would take the first observation and the second observation, and would stop and not inlude the third observation. One other small syntactic python caveat is that we can do negative indexing, or indexing starting at the end of a list. We do this by employing negative indexes. The index '-1' represents the **last** element in a list, '-2' the **second to last** observation, and so on. So turing out attention to the first line of code nelow, the first ':' in the *iloc* function tells the function that we want all rows. For the $X$ variable, the ':-1' as the second argument to the *iloc* function means that we want all columns except the last one. For the $y$ variable, we just enter a '3' for the second argument to the *iloc* function as we want only the third column (for those coming from R or other languages, remember that python, like many programming languages, uses zero-based indexing. This means that the first element in many data structures is indexed by a '0').

In [4]:
X = df.iloc[:, :-1].values
y = df.iloc[:, 3].values
print("X = ", X, '\n')
print("y = ", y, '\n')

X =  [['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 nan]
 ['France' 35.0 58000.0]
 ['Spain' nan 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]] 

y =  ['No' 'Yes' 'No' 'No' 'Yes' 'Yes' 'No' 'Yes' 'No' 'Yes'] 



Now we must take care of missing data. Many modern datasets will lack some data and it is very important to deal with it in an intelligent way. One intuition might be to remove a particular observation if it has any missing data. This is generally not considered a great practice because as datasets get larger, we expect many observations to have some sort of missing data. It seems quite wasteful to remove a perfectly good observation just because one of it features lacks data. For a sparse data set, such a practice could remove the majority of our data set. Because of this, we explore different methods. One logical solution, and the one used here, will be to replace any missing values with the mean of the column. This allows us to get all of the rest of the information from a given row without skewing the data all that much. Now this of course only works numeric columned values (you can't take the mean of a country). There are solutions to filling in missing categorical values, but they will not be explored at this point. Moving forward with our "replace with the average idea", we will turn to an object from the sklearn library, another extremely powerful data analytics python library, called *Imputer*. After importing the necessary method, we create an instance of the *Imputer* object and tell it that we want to use the mean as the method. Next, we fit the object on the columns that have missing data (using the *fit* method), which are the second and third columns (which since we used zero-based indexing means the columns indexed 1,2). Remember that to index these columns, we used '1:3' which means start at the column indexed 1 and end at but do not included the column indexed 3. After we have fit the object, we replace the columns in $X$ with the columns if $X$ after calling the transform method, which is done on the second to last line. In the last line, we print the results and can see that our missing values have indeed been replaced with the average of the column.

In [5]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(X[:, 1:3])
X[:, 1:3] = imputer.transform(X[:, 1:3])
print(X)

[['France' 44.0 72000.0]
 ['Spain' 27.0 48000.0]
 ['Germany' 30.0 54000.0]
 ['Spain' 38.0 61000.0]
 ['Germany' 40.0 63777.77777777778]
 ['France' 35.0 58000.0]
 ['Spain' 38.77777777777778 52000.0]
 ['France' 48.0 79000.0]
 ['Germany' 50.0 83000.0]
 ['France' 37.0 67000.0]]


Now we must encode any categorical variables we might have. When builidng machine learning models, we are training a mathematical model on data, and strings (like 'Country' in this data set) have no meaning as mathematical objects. That is, until we encode them. We will not go over the details of how these object are encoded here, but the general methodology is to created $n-1$ disjoint dummy variables, which can take on values 0 and 1, for your objects. For example, we might have two dummy variables named *spain* and *france*. If *spain* is 1, then the given observation is from Spain. If *france* is 1, the given observation is from France. Since they are disjoint, both *spain* and *france* cannot be 1 at the same time. You might be wondering how Germany is represented since we only have variables for *spain* and *france*. A given observation is from Germany if both *spain* and *france* are 0. Again, these details are secondary as we are more concerned with how to actually do it in code. The process is a bit syntax heavy it will be helpful to spend some time looking over the code, but the process is largely the same as above. Import and create the necessary objects from sklearn, fit those objects on the data that needs encoding (in this case only the first column, or column with index 0 from $X$), then set that column equal to its transformed version. We also do the same this for $y$ since 'Yes' and 'No' also have no meaning in models without encoding. The output does not look pretty since python is outputing high predicision scientific notated versions of our data, but we can see that the encoding has indeed occured. Python does encoding a little differently as it has a binary variable for each of our different kinds of observations, but the idea is the same.

In [6]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Country column
ct = ColumnTransformer([('Country', OneHotEncoder(), [0])], remainder = 'passthrough')
X = ct.fit_transform(X)
print('X = ', X)

labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
print('y = ', y)

X =  [[1.0 0.0 0.0 44.0 72000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 1.0 0.0 30.0 54000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]
 [1.0 0.0 0.0 35.0 58000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [1.0 0.0 0.0 37.0 67000.0]]
y =  [0 1 0 0 1 1 0 1 0 1]


The penultimate step in data preprocessing it to split our data into a training and a testing sets. What this means is that we want to split up our data into two disjoint sets of observations. The 'train' set of this data is what the model is going to learn on. Once the model has learned on the train data, we will use the 'test' set to see how well the model is able to preform. We do this by feeding the features of an observation into our model. The model will output a prediction based on the features and we then compare the prediction given by the model to the actual value of the response for that observation. It is absolutely critical that there is no overlap between these sets. To understand why, consider this scenario. A teacher always gives out a practice exam before the actual exam and wants to know how performance on the practice exam relates to performance on the real exam (the practice exam is our 'train' set and the real exam is our 'test' set). If there is overlap between the questions on the practice exam and the real exam, then the performance on the real exam is going to be biased. If a student studies well on the practice and then sees some of the exact same questions on the real exam, then they are obviously going to get the questions they have already seen right on the final, thus the teachers results are biased and not fully representative of how well the students learned. The obvious next question is "how much of our data should be train and how much should be test?". The answer is: it varies. Maybe not the most concrete answer, but true nonetheless. For a very large dataset, it is possible to get away with a 50-50 split, but the general starting point is 80-20 train-test. It is then up to the person creating the model to tinker with these values to achieve the best result. sklearn provides an extremely easy way to split up into train test, shown below. From the output, we see that $X$ has indeed been split up into two disjoint sets with a ration of 80-20.

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)
print("X train = ", X_train, "\n")
print("X test = ", X_test, "\n")

X train =  [[1.0 0.0 0.0 37.0 67000.0]
 [0.0 0.0 1.0 27.0 48000.0]
 [0.0 0.0 1.0 38.77777777777778 52000.0]
 [1.0 0.0 0.0 48.0 79000.0]
 [0.0 0.0 1.0 38.0 61000.0]
 [1.0 0.0 0.0 44.0 72000.0]
 [1.0 0.0 0.0 35.0 58000.0]] 

X test =  [[0.0 1.0 0.0 30.0 54000.0]
 [0.0 1.0 0.0 50.0 83000.0]
 [0.0 1.0 0.0 40.0 63777.77777777778]] 



A final step for many machine learning models is called feature scaling. The intuition behind feature scaling can get a bit mathematical so not much time will be spent trying to understand it, but it is good practice. The idea behind feature scaling is that if variables are on extremely different scales, then the model may become biased. Take for example our Age and Salary features. Age ranges from 27 to 50 while salary 48000 to 83000. If we were to create a model out of this, the term for Salary would dominate the term for Age because the values for Salary are just so much larger and have so much larger of a scale than Age. The remedy is to scale all values to have roughly the same range. An example scaling procedure might be $x_{scale}=\frac{x_{obs} - \mu_{x}}{\sigma_{x}}$ where $\mu_{x}$ is the mean of the $x$ values and $\sigma_{x}$ is the standard deviation of $x$. For those who have taken statistics courses, this should look very similar to the z-value or t-value in hypothesis testing, which often fall between -4 and 4. There are many other scaling methods and which one is used is largely a concern, though one can tinker with different ones to see which works best. Here, we will just let sklearn do it for us. The procedure is again similar to what we have already seen so looking through the code below should be sufficient to understand what is happening. From the output, we can see that our values for Age and Salary have indeed been scaled to have a common range. Note that there really isn't a need to scale binary variables though it won't hurt if one does it. 

In [8]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
X_train = sc_X.fit_transform(X_train)
X_test = sc_X.transform(X_test)
print("X train = ", X_train)

X train =  [[ 0.8660254   0.         -0.8660254  -0.2029809   0.44897083]
 [-1.15470054  0.          1.15470054 -1.82168936 -1.41706417]
 [-1.15470054  0.          1.15470054  0.08478949 -1.0242147 ]
 [ 0.8660254   0.         -0.8660254   1.5775984   1.62751925]
 [-1.15470054  0.          1.15470054 -0.04111006 -0.14030338]
 [ 0.8660254   0.         -0.8660254   0.93011502  0.94003267]
 [ 0.8660254   0.         -0.8660254  -0.52672259 -0.43494049]]


We have now seen a general overview of how to preprocess data for machine learning models. It is critical to undergo these steps as needed before starting any sort of machine learning model to ensure the accuracy and integrity of the model created. 